In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt 
%matplotlib inline


In [2]:
# Import the Rotten Tomatoes bestofrt TSV file into a DataFrame
df = pd.read_csv("./data/bestofrt.tsv", sep='\t')

# Check to see if the file was imported correctly
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [3]:
import requests

In [4]:
url = "https://rottentomatoes.com/m/et_the_extraterrestrial"

response = requests.get(url)

In [5]:
# Save HTML to file
with open("et_the_extraterrestrial.html", mode='wb') as file:
    file.write(response.content)

In [6]:
# Work with HTML in memory
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content)

In [7]:
import zipfile
with zipfile.ZipFile('./data/rt-html.zip', 'r') as myzip:
    myzip.extractall('./data/')

In [8]:
with open('./data/rt_html/et_the_extraterrestrial.html') as file:
    soup = BeautifulSoup(file)

### The task is to extract the title, audience score, and number of audience ratings in each HTML file so each trio can be appended as a dictionary to df_list.

In [9]:
# to find title
soup.find('title') # to find the tag
title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')] # to find the content of the tag

In [10]:
# to find audience score
with open('./data/rt_html/1012007-laura.html') as file:
    soup = BeautifulSoup(file)
    audience_score = soup.find_all(attrs={"class": "superPageFontColor", "style":"vertical-align:top"})[0].contents[0][:-1]
    number_of_audience_ratings = soup.find_all(attrs={"class": "subtle superPageFontColor"})
    print(int(audience_score)/100)

0.91


In [11]:
# to find number of audience ratings
number_of_audience_ratings = soup.find_all("div",class_= "audience-info hidden-xs superPageFontColor")[0].find_all("div")[1].contents[2][len("\n        "):].replace(',', '')
int(number_of_audience_ratings)

10481

In [13]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = './data/rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        # Your code here
        # Note: a correct implementation may take ~15 seconds to run
        soup = BeautifulSoup(file, 'lxml')
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
        audience_score = soup.find_all(attrs={"class": "superPageFontColor", "style":"vertical-align:top"})[0].contents[0][:-1]
        num_audience_ratings = soup.find_all("div",class_= "audience-info hidden-xs superPageFontColor")[0].find_all("div")[1].contents[2][len("\n        "):].replace(',', '')
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])
df.head()

,title,audience_score,number_of_audience_ratings
0,12 Angry Men (Twelve Angry Men) (1957),97,103672
1,The 39 Steps (1935),86,23647
2,The Adventures of Robin Hood (1938),89,33584
3,All About Eve (1950),94,44564
4,All Quiet on the Western Front (1930),89,17768


In [25]:
df_visual = pd.read_csv('./data/bestofrt.tsv', sep='\t')
plt.scatter(df.audience_score, df.cr)

In [26]:
df_visual.columns

Index(['ranking', 'critic_score', 'title', 'number_of_critic_ratings'], dtype='object')